In [3]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
import tensorflow as tf

In [26]:
#PARAMETERS

# how much should we take each learned data
# too large, might skip optimal, to small takes too much time/data/iterations
learning_rate = 0.01

# 
training_iteration = 30

#
batch_size = 100

#
display_step = 2

In [6]:
# Create main model
# model = dataflow graph
# nodes in model = nodes , nodes = units of computation

# each node takes in a tensor and outputs a tensor
# tensor is the data rep, multi dimensional array

In [7]:
#input, one for each pixel
x = tf.placeholder("float", [None, 784])

# ouput, one for each class
y = tf.placeholder('float', [None, 10])

In [9]:
# Create model

# Weight are updated during training - graph weights
W = tf.Variable(tf.zeros([784, 10]))

# Bias shifts the regression line
b = tf.Variable(tf.zeros([10]))

In [10]:
# helps us visualize the data
with tf.name_scope("Wx_b") as scope:
    # linear regression model
    model = tf.nn.softmax(tf.matmul(x, W)+b)

In [11]:
# summary operations to vizualize and collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [13]:
# help minimize error during training
# here we use cross entropy
with tf.name_scope("cost_function") as scope:
    cost_function = -tf.reduce_sum(y*tf.log(model))
    
    # scalar summary to monitor cost function
    tf.summary.scalar("cost_function", cost_function)

In [14]:
with tf.name_scope("train") as scope:
    # gradient descent is paced by the learning rate
    # and minimized with cost_function
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [18]:
init = tf.global_variables_initializer()

In [19]:
merged_summary_op = tf.summary.merge_all()

In [31]:
# Lauching graph session
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter('.', graph_def=sess.graph)
    
    # training cycle
    for iteration in range(training_iteration):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # loop over batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # fir training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys})
            
            # Compute average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs,y: batch_ys})/total_batch

            # write to summary for tf board
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch+i)

        # pritns logs
        if iteration % display_step == 0:
            print("iteration", '%04d' % (iteration+1), "cost", "{:.9f}".format(avg_cost))
            
    print("training complete")
    
    # check how model values measure up against test values
    predictions = tf.equal(tf.arg_max(model, 1), tf.arg_max(y, 1))
    
    # mean accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("accuracy", accuracy.eval({x:mnist.test.images, y: mnist.test.labels}))

iteration 0001 cost 29.860465115
iteration 0003 cost 21.126813491
iteration 0005 cost 20.324054164
iteration 0007 cost 19.629978932
iteration 0009 cost 19.336096870
iteration 0011 cost 19.083457223
iteration 0013 cost 18.995934640
iteration 0015 cost 18.749809804
iteration 0017 cost 18.580422345
iteration 0019 cost 18.518634330
iteration 0021 cost 18.481242117
iteration 0023 cost 18.462068937
iteration 0025 cost 18.250765198
iteration 0027 cost 18.113451968
iteration 0029 cost 18.128990116
training complete
accuracy 0.9218


$ tensorboard --logdir=.